In [ ]:
! Pip install Pandas

In [ ]:
! Pip install Numpy

## Importing the Necessary Tools

In [ ]:
import pandas as pd
import numpy as np

### Extracting Layer

In [ ]:
yanki_df = pd.read_csv(r'data_set\raw_data\yanki_ecommerce.csv')

In [ ]:
yanki_df.head()

In [ ]:
yanki_df.info()

In [ ]:
yanki_df.columns

In [ ]:
#drop missing values
yanki_df.dropna (subset=['Order_ID', 'Customer_ID'], inplace=True)

In [ ]:
yanki_df.info()

In [ ]:
# Convert Order_date from string to datetime object
yanki_df['Order_Date'] = pd.to_datetime(yanki_df['Order_Date'])

In [ ]:
# customer table
customer_df = yanki_df[['Customer_ID', 'Customer_Name', 'Email', 'Phone_Number']].copy().drop_duplicates().reset_index(drop=True)

In [ ]:
customer_df.head()

In [ ]:
products_df = yanki_df[['Product_ID', 'Product_Name', 'Brand', 'Category','Price']].copy().drop_duplicates().reset_index(drop=True)
products_df.head()

In [ ]:
yanki_df.columns

In [ ]:
# shipping_address_df
shipping_address_df = yanki_df[['Customer_ID', 'Shipping_Address', 'City', 'State', 'Country', \
                                'Postal_Code']].copy().drop_duplicates().reset_index(drop=True)
shipping_address_df.index.name = 'shipping_ID'
shipping_address_df = shipping_address_df.reset_index()
shipping_address_df.head()

In [ ]:
# Order_df
orders_df = yanki_df[['Order_ID', 'Customer_ID', 'Product_ID', 'Quantity', 'Total_Price', \
                        'Order_Date']].copy().drop_duplicates().reset_index(drop=True)
orders_df.head()

In [ ]:
#payment_method_df
payment_method_df = yanki_df[['Order_ID', 'Payment_Method', 'Transaction_Status']].copy().drop_duplicates().reset_index(drop=True)

payment_method_df.head()

In [ ]:
# save the tables to cavs
customer_df.to_csv(r'data_set/clean_data/customers.csv', index=False)
products_df.to_csv(r'data_set/clean_data/products.csv', index=False)
orders_df.to_csv(r'data_set/clean_data/orders.csv', index=False)
shipping_address_df.to_csv(r'data_set/clean_data/shipping_address.csv', index=False)
payment_method_df.to_csv(r'data_set/clean_data/payment_method.csv', index=False)

In [ ]:
! pip install psycopg2

In [ ]:
import psycopg2

In [ ]:
def get_db_connection():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'yanki_ecommerce',
        user= 'postgres',
        password= 'password'
    )
    return connection

In [ ]:
conn = get_db_connection()

In [ ]:
# Create our SQL tables
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''
                            CREATE SCHEMA IF NOT EXISTS yanki;
                            DROP TABLE IF EXISTS yanki.customers CASCADE;
                            DROP TABLE IF EXISTS yanki.products CASCADE;
                            DROP TABLE IF EXISTS yanki.shipping_address CASCADE;
                            DROP TABLE IF EXISTS yanki.orders CASCADE;
                            DROP TABLE IF EXISTS yanki.payment_method CASCADE;
                            
                            CREATE TABLE IF NOT EXISTS yanki.customers (
                                Customer_ID UUID PRIMARY KEY,
                                Customer_Name TEXT,
                                Email TEXT,
                                Phone_Number TEXT
                            );

                            CREATE TABLE IF NOT EXISTS yanki.products (
                                Product_ID UUID PRIMARY KEY,
                                Product_Name TEXT,
                                Brand TEXT,
                                Category TEXT,
                                Price FLOAT
                                );

                            CREATE TABLE IF NOT EXISTS yanki.shipping_address (
                                shipping_ID SERIAL PRIMARY KEY,
                                Customer_ID UUID,
                                Shipping_Address TEXT,
                                City TEXT,
                                State TEXT,
                                Country TEXT,
                                Postal_Code INTEGER,
                                FOREIGN KEY(Customer_ID) REFERENCES yanki.customers(Customer_ID)
                                );
                            
                            CREATE TABLE IF NOT EXISTS yanki.orders (
                                Order_ID UUID PRIMARY KEY,
                                Customer_ID UUID,
                                Product_ID UUID,
                                Quantity INTEGER,
                                Total_Price FLOAT,
                                Order_Date DATE,
                                FOREIGN KEY (Customer_ID) REFERENCES yanki.customers (Customer_ID),
                                FOREIGN KEY (Product_ID) REFERENCES yanki.products (Product_ID)
                            );


                            CREATE TABLE IF NOT EXISTS yanki.payment_method (
                                Order_ID UUID,
                                Payment_Method TEXT,
                                Transaction_Status TEXT,
                                FOREIGN KEY (Order_ID) REFERENCES yanki.orders (Order_ID)
                            );'''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

In [ ]:
create_tables()

In [ ]:
display (products_df.columns)
display (products_df.info())

In [ ]:
import csv
def load_data_from_csv(csv_path):
    conn=get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute('''
                                INSERT INTO yanki.customers (Customer_ID, Customer_Name, Email, Phone_Number)
                                VALUES (%s, %s, %s, %s);''',
                                row
            )
    conn.commit()
    cursor.close()
    conn.close()
csv_file_path = r'data_set\clean_data\customers.csv'
load_data_from_csv(csv_file_path)

In [ ]:
import csv
def load_data_from_csv(csv_path):
    conn=get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute('''
                                INSERT INTO yanki.products (Product_ID, Product_Name, Brand, Category, Price)
                                VALUES (%s, %s, %s, %s, %s);''',
                                row
            )
    conn.commit()
    cursor.close()
    conn.close()
csv_file_path = r'data_set\clean_data\products.csv'
load_data_from_csv(csv_file_path)

In [ ]:
import csv
def load_data_from_csv(csv_path):
    conn=get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute('''
                                INSERT INTO yanki.shipping_address (shipping_ID,Customer_ID,  Shipping_Address, City, State, Country, Postal_Code)
                                VALUES (%s, %s, %s, %s, %s, %s, %s);''',
                                row
            )
    conn.commit()
    cursor.close()
    conn.close()
csv_file_path = r'data_set\clean_data\shipping_address.csv'
load_data_from_csv(csv_file_path)

In [ ]:
import csv
def load_data_from_csv(csv_path):
    conn=get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute('''
                                INSERT INTO yanki.orders (Order_ID, Customer_ID, Product_ID, Quantity,Total_Price, Order_Date)
                                VALUES (%s, %s, %s, %s, %s, %s);''',
                                row
            )
    conn.commit()
    cursor.close()
    conn.close()
csv_file_path = r'data_set\clean_data\orders.csv'
load_data_from_csv(csv_file_path)

In [ ]:
import csv
def load_data_from_csv(csv_path):
    conn=get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute('''
                                INSERT INTO yanki.payment_method (Order_ID, Payment_Method, Transaction_Status)
                                VALUES (%s, %s, %s);''',
                                row
            )
    conn.commit()
    cursor.close()
    conn.close()
csv_file_path = r'data_set\clean_data\payment_method.csv'
load_data_from_csv(csv_file_path)

In [ ]:
payment_method_df.info() 